# Obteniendo el TOTAL de casos positivos, fallecidos y vacunados contra COVID-19 por cada departamento del Perú

En este notebook se busca obtener un dataset con el TOTAL con el número de **casos positivos**, **fallecidos** y **totalmente vacunados** por cada **departamento** del Perú. De la misma manera se agrega el **número de habitantes** de cada departamento, para calcular la **tasa de mortalidad por 100 000 habitantes** y el **porcentaje de vacunación** respectivo.

Se considera como **completamente vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

En esta sección se lee y procesa el dataset **Data Product 1** ([DP1_covid19-peru_x_semanaEpi.csv](Data/DP1_covid19-peru_x_semanaEpi.csv)) para obtener un resumen general por departamento.

Datos de población estimada para el 2020 obtenida del reporte [«PERÚ:Estimaciones y Proyecciones de Población por Departamento, Provincia y Distrito, 2018 - 2020»](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1715/Libro.pdf). Página 39 estimaciones de población total proyectada para 2020. Fuente: [INEI](https://www.inei.gob.pe/).

In [11]:
import pandas as pd
import dask.dataframe as dd

In [12]:
# Cargamos un diccionario con el número de habitantes por departamento en 2020
dic_dep = {
    "AMAZONAS"	:   426806,
    "ANCASH"	:   1180638,
    "APURIMAC"	:   430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:   668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LA LIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADRE DE DIOS":173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SAN MARTIN":	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
}

# Convertimos a dataframe
df_dep = pd.DataFrame(dic_dep.items(), columns = ['departamento', 'no_habitantes'])

In [13]:
# Leer el DP1
url_dp1 = '../Data/DP1_covid19-peru_x_semanaEpi.csv'
df = pd.read_csv(url_dp1)

# Procesar el df (sumar cada col)
df = pd.DataFrame(df.sum(axis = 0, skipna = True)).reset_index().rename({'index': 'var', 0: 'total'}, axis=1)

# Dejar solamente filas que contengan el nombre de un departamento
lst_dfs = [] 

for dep in df_dep['departamento']:
    temp_df = df.loc[df['var'].str.contains(dep)]
    if not temp_df.empty:
        lst_dfs.append(temp_df)
        
df = pd.concat(lst_dfs, ignore_index=True).drop_duplicates()  # Drop duplicates

# Eliminamos a vacunados (_vac) porque necesitamos todas las dosis 1, 2 y 3 no solo vacunación completa
df = df[~df['var'].str.contains('_vac')]  
    
# Separate in sub dfs and remove substrings
df_cas = df[df['var'].str.contains('_cas')].rename({'var': 'departamento','total': 'casos'}, axis = 1)
df_cas['departamento'] = df_cas['departamento'].str.replace('_cas', '')
df_fal = df[df['var'].str.contains('_fal')].rename({'var': 'departamento','total': 'fallecidos'}, axis = 1)
df_fal['departamento'] = df_fal['departamento'].str.replace('_fal', '')

# Unir en un solo dataframe el resumen por departamentos 
df_summary = df_dep.merge(df_cas, how='left', on='departamento')
df_summary = df_summary.merge(df_fal, how='left', on='departamento')

df_summary

,departamento,no_habitantes,casos,fallecidos
0,AMAZONAS,426806,34304,1293
1,ANCASH,1180638,89671,6789
2,APURIMAC,430736,28121,1528
3,AREQUIPA,1497438,132447,9777
4,AYACUCHO,668213,37397,2175
5,CAJAMARCA,1453711,73228,4203
6,CALLAO,1129854,115210,10153
7,CUSCO,1357075,89454,4855
8,HUANCAVELICA,365317,17586,1186
9,HUANUCO,760267,39618,2738


## B. Agregar el número de dosis

In [14]:
# Leer solamente "id_eess" y "dosis" del dataset de vacunados
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
vac_col = ['id_eess', 'dosis']
df_vac = dd.read_csv(url_vac, sep = ",", usecols = vac_col, dtype = {'id_eess':'int32','dosis': 'category'})

# Obtener las variables dummies de la variable "dosis"
df_vac = df_vac.categorize()
dosis = dd.get_dummies(df_vac['dosis']).drop(['4', '5'], axis = 1)

# Ahora se tiene la ubicación y el número de dosis aplicada
df_vac = dd.concat([df_vac, dosis], axis=1, ignore_unknown_divisions=True)
del dosis, df_vac['dosis']

In [15]:
# Leer los datasets de ubigeo y centros de vacunación
url_ubigeo = '../RawData/TB_UBIGEOS.csv'
url_vaccenter = '../RawData/TB_EESS.csv'
ubigeo = pd.read_csv(url_ubigeo, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(url_vaccenter, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante "id_ubigeo"
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

In [16]:
# Encontrar el departamento correspondiente de todas las personas vacunadas
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')

# Contar el total de número de dosis por departamento
df_vac = df_vac.groupby('departamento').agg({'1': ['sum'], '2': ['sum'], '3': ['sum']}).compute()
df_vac = df_vac.astype({('1', 'sum'): 'int64', ('2', 'sum'): 'int64', ('3', 'sum'): 'int64'})

In [17]:
# Unir al df resumen (df_summary)
df_summary = df_summary.merge(df_vac, how='left', on='departamento')

# Rename cols
df_summary.rename(columns = {'index':'departamentos', ('1', 'sum'):'dosis_1', 
                             ('2', 'sum'): 'dosis_2', ('3', 'sum'):'dosis_3'}, inplace = True)

df_summary

,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3
0,AMAZONAS,426806,34304,1293,267803,231797,54467
1,ANCASH,1180638,89671,6789,923951,867235,265429
2,APURIMAC,430736,28121,1528,347658,305790,66954
3,AREQUIPA,1497438,132447,9777,1174518,1077591,291509
4,AYACUCHO,668213,37397,2175,438764,357984,51380
5,CAJAMARCA,1453711,73228,4203,1048324,917081,150325
6,CALLAO,1129854,115210,10153,891624,849937,338551
7,CUSCO,1357075,89454,4855,994634,869726,152123
8,HUANCAVELICA,365317,17586,1186,248316,199252,30306
9,HUANUCO,760267,39618,2738,530300,446026,89879


## C. Añadir detalles

In [18]:
# Mortality rate per 100k per department
df_summary['tasa_mortalidad'] = round((df_summary['fallecidos']/df_summary['no_habitantes'])*100000,2)

# % of people fully vaccinated per department
df_summary['vac_porcentaje'] = round((df_summary['dosis_2']*100)/df_summary['no_habitantes'],2)

df_summary

,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3,tasa_mortalidad,vac_porcentaje
0,AMAZONAS,426806,34304,1293,267803,231797,54467,302.95,54.31
1,ANCASH,1180638,89671,6789,923951,867235,265429,575.03,73.45
2,APURIMAC,430736,28121,1528,347658,305790,66954,354.74,70.99
3,AREQUIPA,1497438,132447,9777,1174518,1077591,291509,652.92,71.96
4,AYACUCHO,668213,37397,2175,438764,357984,51380,325.50,53.57
5,CAJAMARCA,1453711,73228,4203,1048324,917081,150325,289.12,63.09
6,CALLAO,1129854,115210,10153,891624,849937,338551,898.61,75.23
7,CUSCO,1357075,89454,4855,994634,869726,152123,357.75,64.09
8,HUANCAVELICA,365317,17586,1186,248316,199252,30306,324.65,54.54
9,HUANUCO,760267,39618,2738,530300,446026,89879,360.14,58.67


In [19]:
# Resumen general
print(f'Total habitantes: {df_summary.no_habitantes.sum()}')
print(f'Total fallecidos: {df_summary.fallecidos.sum()}')
print(f'Total vacunados con 1 dosis: {df_summary.dosis_1.sum()}')
print(f'Total vacunados con 2 dosis: {df_summary.dosis_2.sum()}')
print(f'Total vacunados con 3 dosis: {df_summary.dosis_3.sum()}')
print(f'Porcentaje de la población vacunada: {round(df_summary.dosis_2.sum()*100/df_summary.no_habitantes.sum(),2)}')

Total habitantes: 32625948
Total fallecidos: 203376
Total vacunados con 1 dosis: 24786135
Total vacunados con 2 dosis: 22271123
Total vacunados con 3 dosis: 5994692
Porcentaje de la población vacunada: 68.26


## D. Guardar el dataset

In [20]:
df_summary.to_csv('../Data/DP2_covid19-peru_resumen_x_departamentos.csv',index = False)